<a href="https://colab.research.google.com/github/arifmhassan/Spark-Python-Data-Ingestion/blob/master/RetailSales_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aggregations
Primary Data source=online-retail-dataset.csv
purpose: Aggregation using groupby, windows, rollup, sql like dense ranking

Question
*   dataframe read  the data online-retail-dataset.csv with 5 partition
*   Data analysis count distinct e.g. stockcode, min quantity, max quantity, sum of quantity, sum of revenue by Country, CUstomer, date, hour  etc

*   Whick date has the maximum  purchase quantity using dense_rank in sql and python
*   group by accross country/date/ quantity using rollup and cube
*    group by accross country/date/ quantity using pivot to get country listing in a column



In [1]:
# initialize spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz 
!tar xf spark-2.4.5-bin-hadoop2.7.tgz 
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
# Source  git branch clone

%%bash
#-b my-branch
git clone -b master https://github.com/databricks/Spark-The-Definitive-Guide.git master

Cloning into 'master'...
Checking out files: 100% (1738/1738), done.


In [ ]:
# check data quality
%cat master/data/retail-data/all/online-retail-dataset.csv | head -n 5

In [110]:
# read the file
filename="master/data/retail-data/all/online-retail-dataset.csv"
dataframe=spark.read.format("csv").option("header","true").option("inferSchema","true").load(filename).coalesce(5)

In [74]:
#display data 
#create view
dataframe.show(5)
dataframe.createOrReplaceTempView("retailsales")


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [45]:
# Data Analysis
# query data sets
spark.sql("select * from retailsales where Country like '%United%' order by Country asc limit 2").show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|             Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------------+
|   543911|    21485|RETROSPOT HEART H...|       6|2/14/2011 12:46|     4.95|     17829|United Arab Emirates|
|   543911|    48138|  DOORMAT UNION FLAG|       2|2/14/2011 12:46|     7.95|     17829|United Arab Emirates|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------------+



In [5]:
dataframe.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
# Data Analysis Summary
print(dataframe.select ("Country").count())
print(spark.sql("select count( Country) Country  from retailsales").show())

# distinct 
print(dataframe.select ("Country").distinct ().count())
print(spark.sql("select count (distinct Country) from retailsales").show())

# first and last , max, min 
from pyspark.sql.functions import first, last, min, max, countDistinct
print(dataframe.select(first("Country"), last("Country"), first("InvoiceDate"), last("InvoiceDate"),min("InvoiceDate"),max("InvoiceDate"), countDistinct("InvoiceNo")).show())

print(spark.sql("select first( Country), last(Country),first(InvoiceDate), last(InvoiceDate) , max(InvoiceDate), min(InvoiceDate), count(distinct InvoiceNo)  from retailsales").show())




In [ ]:
#sum of quantity, sum of revenue by Country, CUstomer, date, hour etc
from pyspark.sql.functions import  sum
dataframe.groupBy("Country").agg(    sum("Quantity").alias("Total Revenue")).show()
spark.sql("select Country, sum(Quantity) from retailsales group by Country").show()


In [111]:
# Change datatype to date YYYY mm dd hh mm

from pyspark.sql.functions import unix_timestamp, from_unixtime
dataframe=dataframe.withColumn("InvoiceDate",from_unixtime(unix_timestamp('InvoiceDate', 'MM/dd/yyy hh:mm')))

In [113]:
# extract hour
from pyspark.sql.functions import hour
dataframe=dataframe.select("*",hour("InvoiceDate").alias("Hour"))


In [ ]:
from pyspark.sql.functions import  sum
dataframe.groupBy("Hour").agg(    sum("Quantity").alias("Total Revenue")).show()
#spark.sql("select Country, sum(Quantity) from retailsales group by Country").show()

In [132]:
from pyspark.sql.functions import month, quarter
from pyspark.sql.functions import sum
#dataframe.groupBy("InvoiceDate").agg(sum("Quantity")).show()
dataframe.groupBy(month("InvoiceDate")).agg(sum("Quantity")).show()

+------------------+-------------+
|month(InvoiceDate)|sum(Quantity)|
+------------------+-------------+
|                12|       271213|
|              null|      2427623|
|                 1|       179512|
|                 6|       155500|
|                 3|       199155|
|                 5|       210883|
|                 9|       294181|
|                 4|       171310|
|                 8|       207967|
|                 7|       181257|
|                10|       338270|
|                11|       356742|
|                 2|       182837|
+------------------+-------------+

